In [1]:
%matplotlib inline
import pandas as pd
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# Headcount assumptions

In numbers of staff members

In [2]:
groups = ["Data", "Eng", "HPC", "Math"]

In [3]:
hourly = pd.Series(None, index=groups)
hourly["Data"] = 13
hourly["Eng"] = 1
hourly["HPC"] = 9
hourly["Math"] = 2

In [4]:
regular = pd.Series(None, index=groups)
regular["Data"] = 39
regular["Eng"] = 9
regular["HPC"] = 26
regular["Math"] = 23

In [5]:
total = hourly + regular

# FY17 requests

In FTE units

## Estimated management costs

Including group manager and team leads

In [6]:
fy17_mgmt = pd.Series(None, index=groups)
fy17_mgmt["Data"] = 0.5 + 4*0.20
fy17_mgmt["Eng"] = 0.25
fy17_mgmt["HPC"] = 0.35 + 3*0.15
fy17_mgmt["Math"] = 0.35 + 2*0.15
fy17_mgmt

Data    1.30
Eng     0.25
HPC     0.80
Math    0.65
dtype: float64

## Estimated admin costs

In [7]:
fy17_admin = pd.Series(None, index=groups)
fy17_admin["Data"] = 0.70
fy17_admin["Eng"] = 0.15
fy17_admin["HPC"] = 0.5
fy17_admin["Math"] = 0.35

In [8]:
df = pd.DataFrame({"staff" : total, "regular" : regular, "hourly" : hourly, "admin" : fy17_admin, "mgmt" : fy17_mgmt})
df

,admin,hourly,mgmt,regular,staff
Data,0.70,13.0,1.30,39.0,52.0
Eng,0.15,1.0,0.25,9.0,10.0
HPC,0.50,9.0,0.80,26.0,35.0
Math,0.35,2.0,0.65,23.0,25.0


# Admin support

In [9]:
df["admin"]

Data    0.70
Eng     0.15
HPC     0.50
Math    0.35
Name: admin, dtype: float64

## Currently planned per-capita admin support

In [10]:
df["admin"] / df["staff"]

Data    0.013462
Eng     0.015000
HPC     0.014286
Math    0.014000
dtype: float64

## Evenly distributed admin support

In [11]:
fit1 = sm.ols(formula="admin ~ staff", data=df).fit()
fit1.summary()

C:\Users\bake113\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admin   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     938.2
Date:                Sun, 10 Jul 2016   Prob (F-statistic):            0.00106
Time:                        17:09:25   Log-Likelihood:                 13.037
No. Observations:                   4   AIC:                            -22.07
Df Residuals:                       2   BIC:                            -23.30
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0229      0.015      1.561      0.259        -0.040     0.086
staff          0.0132      0.000     30.631      0.001         0.011     0.015
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.200
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.736
Skew:                           0.974   Prob(JB):                        0.692
Kurtosis:                       2.209   Cond. No.                         76.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
fit1.fittedvalues

Data    0.708441
Eng     0.154743
HPC     0.484325
Math    0.352492
dtype: float64

In [13]:
fit2 = sm.ols(formula="admin ~ regular + hourly", data=df).fit()
fit2.summary()

C:\Users\bake113\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admin   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     273.4
Date:                Sun, 10 Jul 2016   Prob (F-statistic):             0.0427
Time:                        17:09:25   Log-Likelihood:                 13.343
No. Observations:                   4   AIC:                            -20.69
Df Residuals:                       1   BIC:                            -22.53
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0301      0.026      1.151      0.455        -0.302     0.363
regular        0.0125      0.002      6.881      0.092        -0.011     0.036
hourly         0.0147      0.004      3.789      0.164        -0.035     0.064
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.622
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.464
Skew:                           0.394   Prob(JB):                        0.793
Kurtosis:                       1.529   Cond. No.                         84.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
fit2.fittedvalues

Data    0.708671
Eng     0.157211
HPC     0.487404
Math    0.346714
dtype: float64

# Management support

In [15]:
df["mgmt"]

Data    1.30
Eng     0.25
HPC     0.80
Math    0.65
Name: mgmt, dtype: float64

## Currently planned per-capita management support

In [16]:
df["mgmt"] / df["staff"]

Data    0.025000
Eng     0.025000
HPC     0.022857
Math    0.026000
dtype: float64

## Evenly distributed management support

In [17]:
fit1 = sm.ols(formula="mgmt ~ staff", data=df).fit()
fit1.summary()

C:\Users\bake113\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mgmt   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     207.9
Date:                Sun, 10 Jul 2016   Prob (F-statistic):            0.00478
Time:                        17:09:25   Log-Likelihood:                 7.5458
No. Observations:                   4   AIC:                            -11.09
Df Residuals:                       2   BIC:                            -12.32
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0030      0.058      0.052      0.963        -0.246     0.252
staff          0.0245      0.002     14.419      0.005         0.017     0.032
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.479
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.606
Skew:                          -0.828   Prob(JB):                        0.739
Kurtosis:                       2.056   Cond. No.                         76.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
fit1.fittedvalues

Data    1.276554
Eng     0.247937
HPC     0.860209
Math    0.615300
dtype: float64

In [19]:
fit2 = sm.ols(formula="mgmt ~ regular + hourly", data=df).fit()
fit2.summary()

C:\Users\bake113\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mgmt   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     112.1
Date:                Sun, 10 Jul 2016   Prob (F-statistic):             0.0666
Time:                        17:09:25   Log-Likelihood:                 9.0733
No. Observations:                   4   AIC:                            -12.15
Df Residuals:                       1   BIC:                            -13.99
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.0522      0.076     -0.686      0.617        -1.019     0.914
regular        0.0299      0.005      5.661      0.111        -0.037     0.097
hourly         0.0125      0.011      1.104      0.468        -0.131     0.156
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.622
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.464
Skew:                          -0.394   Prob(JB):                        0.793
Kurtosis:                       1.529   Cond. No.                         84.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
fit2.fittedvalues

Data    1.274787
Eng     0.229033
HPC     0.836626
Math    0.659555
dtype: float64